In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('./csv/df_stock_temp_reset.csv', index_col=0)
df1.head()

,작성일,제목,적정가격,투자의견,작성자,제공출처,uri
0,2021-11-03,카카오뱅크(323410)포트폴리오 급변 시기에는 순마진율을 봐야,0,Not Rated,김도하,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
1,2021-11-03,삼성SDI(006400)EV 향 원형전지가 효자,"860,000",Buy,이순학,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
2,2021-11-03,랩지노믹스(084650)[3Q21 Review 2020 년 연간 실적을 넘어서는 ...,"57,000",Buy,김형수,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
3,2021-11-03,카카오뱅크(323410)펀더멘탈은 변함이 없다,"81,000",Buy,김현기,하이투자증권,http://consensus.hankyung.com/apps.analysis/an...
4,2021-11-03,삼성SDI(006400)실적 성장세는 계속된다,"1,000,000",Buy,정원석,하이투자증권,http://consensus.hankyung.com/apps.analysis/an...


### PDF 저장

In [ ]:
# PDF저장 진행 중 connection error 발생
# ProtocolError: ("Connection broken: ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None)
# ", ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
# incompleteRead, FileNotFoundError, InvalidURL
# 1. header추가
# 2. time.sleep()
# 3. random한 time.sleep()
# 4. except 처리
# 시간당 3000개 pdf파일 저장 -> 9000개 3시간
# 30000개 10시간30분
# index 8599 2021-01-19 삼양식품까지 테스트 작업
# uri = 'http://consensus.hankyung.comjavascript:void(0);' 387개
from http.client import IncompleteRead as http_incompleteRead
from urllib3.exceptions import IncompleteRead as urllib3_incompleteRead
import requests
import re
import os
import time
import random

start = time.time()
# pdf저장 경로 지정
if not os.path.isdir('./pdf1/'):
    os.mkdir('./pdf1/')
pdf_path = './pdf1/'

# df1의 인덱스는 0~159636
for i in df1.index[:30000]:
    try : 
        
        headers = {
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
        }
        response = requests.get(df1['uri'][i], headers=headers)
        print(i)
        
        # 파일경로/작성일_제목_고유넘버.pdf 형식으로 저장
        with open( pdf_path + ''.join(df1['작성일'][i].split('-'))+'_'+ \
                  # 제목에 특수문자를 제거하지 않으면 파일 저장 시 pdf파일이 아닌 
                  # 0KB의 빈 파일로 저장됨, 파일이 지워지지 않아 고생
                  re.sub('[^A-Za-z0-9가-힣_-]','',df1['제목'][i].split('(')[0])+'_'+ \
                  df1['uri'][i].split('=')[1] + '.pdf', 'wb') as f:
            f.write(response.content)
            
    except (requests.exceptions.ConnectionError, FileNotFoundError, http_incompleteRead, \
    urllib3_incompleteRead, requests.exceptions.InvalidURL) as e :
        print(e)
        
        #에러 목록 텍스트파일로 저장
        with open('error_list.txt', 'a') as f: 
            f.writelines(str([i, e])+'\n')

print("time :", time.time() - start)


### PDF Parsing

#### Tika

In [8]:
# pip install tika
# Java runtime 설치되어 있어야 함
# local주소에 있는 pdf파일 텍스트 추출
# 약 9000개 22분
# 정상 출력되지 않는 파일 확인 -> NH투자증권만 제대로 출력안됨
import time
import os
from tika import parser

start = time.time()
if not os.path.isdir('./text1/'):
    os.mkdir('./text1/')

# pdf파일이 있는 경로를 반복list로 설정
pdf_list = os.listdir('./pdf1/') ### pdf파일 저장된 경로 설정
for file in pdf_list[:10]:
    text = parser.from_file('./pdf1/{}'.format(file)) ### pdf파일 저장된 경로 설정
    text['content']=str(text['content']).replace('\n', '')
    text['content']
    file = file.replace('.pdf','')
    with open('./text1/{}.txt'.format(file), 'w', encoding='utf-8') as f:
        f.write(text['content'])
print("time :", time.time() - start)

time : 9.635636806488037


#### PDF miner

In [2]:
# 텍스트 변환 함수
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import time
import os
import re

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text.replace('\n','')

In [11]:
# 
# tika와 비교해 파싱속도가 약 40배정도 난다
# 큰 파일들을 대상으로 했을 경우에는 더 차이가 난다.
start = time.time()
if not os.path.isdir('./text1/'):
    os.mkdir('./text1/')

# pdf파일이 있는 경로를 반복list로 설정
pdf_list = os.listdir('./pdf1/') ### pdf파일 저장된 경로 설정
for file in pdf_list[:10]:
    text = convert_pdf_to_txt('./pdf1/'+file)
    file = file.replace('.pdf','')
    with open('./text1/{}.txt'.format(file), 'w', encoding='utf-8') as f:
        f.write(text)
print("time :", time.time() - start)

time : 488.9361255168915


In [5]:
pdf_list = os.listdir('./pdf1/')
pdf_list

['20210901_LG화학_597175.pdf',
 '20210901_S-Oil_597187.pdf',
 '20210901_SKC_597179.pdf',
 '20210901_SK이노베이션_597186.pdf',
 '20210901_교촌에프앤비_597161.pdf',
 '20210901_금호석유_597178.pdf',
 '20210901_대한유화_597181.pdf',
 '20210901_덕산네오룩스_597169.pdf',
 '20210901_롯데정밀화학_597185.pdf',
 '20210901_롯데케미칼_597176.pdf',
 '20210901_메리츠화재_597191.pdf',
 '20210901_브이원텍_597156.pdf',
 '20210901_삼성물산_597167.pdf',
 '20210901_신세계_597148.pdf',
 '20210901_엠로_597193.pdf',
 '20210901_웹케시_597165.pdf',
 '20210901_유진테크_597189.pdf',
 '20210901_코오롱인더_597180.pdf',
 '20210901_한국철강_597170.pdf',
 '20210901_한화솔루션_597177.pdf',
 '20210901_현대백화점_597150.pdf',
 '20210901_호텔신라_597147.pdf',
 '20210901_효성첨단소재_597183.pdf',
 '20210901_효성티앤씨_597182.pdf',
 '20210901_효성화학_597184.pdf',
 '20210902_BNK금융지주_597194.pdf',
 '20210902_KT_597198.pdf',
 '20210902_LG유플러스_597199.pdf',
 '20210902_NAVER_597200.pdf',
 '20210902_SKC_597216.pdf',
 '20210902_SK이노베이션_597217.pdf',
 '20210902_SK텔레콤_597201.pdf',
 '20210902_금화피에스시_597252.pdf',
 '20210902_나스미디어_5972

### 텍스트 파일 전처리

In [ ]:
# 리포트를 작성함에 있어서 관련 규정을 준수하고 있는지를 통지하는 내용들이 
# 리포트 끝에 compliance notice라는 형식으로 적혀있다.
# 분석에 방해가 되는 텍스트이므로 제거해준다.
text_list = os.listdir('./text1')
if not os.path.isdir('./text_remove/'):
    os.mkdir('./text_remove/')
    
# 텍스트에서 compliance notice라는 단어 뒤쪽내용을 자르고 출력하는 함수
def remove_Compliance(file):
    new_text =''
    with open('./text1/{}'.format(file), 'r', encoding='utf-8') as f:
        text= f.read()
        text = re.split("Compliance Notice|Compliance Note|compliance notice|compliance note", text,maxsplit=1)
        new_text = text[0]
    with open('./text_remove/{}'.format(file), 'w', encoding='utf-8') as f:
        f.write(new_text)

In [13]:
# 변환된 텍스트 파일에서 Compliance 단어가 어디쯤 위치해 있는지 찾아서
# 끝쪽에 있는 것만 찾아 제거하기 위해 시도
# 변환된 텍스트 파일의 라인수가 제대로 구분이 안되어 있음
# 아마 줄바꿈 처리가 안돼서 파일을 열어서 보는 라인수와 저장된 라인수가 달라보이는 것 같음
# 간혹 라인수가 많은 것들은 줄바꿈 처리가 되어있는 파일
import re
import os

s = 'Compliance'
p = re.compile(s)

text_list = os.listdir('./text1')
for i in text_list:
    with open('./text1/'+i, 'r', encoding='utf8') as f:
        for x, y in enumerate(f.readlines(),1):
            m = p.findall(y)
            if m:
                print('File %s [ %d ] Line Searching : %s' %(i,x,m))
#                 print('Full Line Text : %s' %y)
        print()

File 20190408_서연이화탐방_537853.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_LG이노텍_528107.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_LG화학_528108.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_LS산전_528139.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_POSCO_528090.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_강원랜드_528038.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_고려아연_528092.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_기아차_528093.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_넷마블_528082.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_대우조선해양_528088.txt [ 1 ] Line Searching : ['Compliance']


File 20190612_만도_528067.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_메디젠휴먼케어_528126.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_메디톡스_528110.txt [ 1 ] Line Searching : ['Compliance']

File 20190612_명진홀딩스_528127.txt [ 1 ] Line Searching : ['Compliance']


File 20190612_삼성SDI_528039

File 20190626_KT_529343.txt [ 1 ] Line Searching : ['Compliance']


File 20190626_NHN한국사이버결제_529332.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_OCI_529473.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_SK네트웍스_529380.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_SK하이닉스_529435.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_ST모티브_529447.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_강원랜드_529436.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_기아차_529441.txt [ 1 ] Line Searching : ['Compliance']


File 20190626_노바텍_529382.txt [ 1 ] Line Searching : ['Compliance']


File 20190626_대덕전자_529479.txt [ 1 ] Line Searching : ['Compliance']


File 20190626_대한항공_529418.txt [ 1 ] Line Searching : ['Compliance']


File 20190626_동원FB_529383.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_디오_529385.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_러셀_529401.txt [ 1 ] Line Searching : ['Compliance']

File 20190626_만도_529407.txt

File 20190703_LG유플러스_530225.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_LG이노텍_530253.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_LG화학_530304.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_NAVER_530234.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_S-Oil_530266.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_SK텔레콤_530198.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_ST모티브_530193.txt [ 1 ] Line Searching : ['Compliance']


File 20190703_대림산업_530320.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_대한해운_530201.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_동원FB_530285.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_롯데정밀화학_530232.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_롯데케미칼_530255.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_매일유업_530286.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_삼성전자_530267.txt [ 1 ] Line Searching : ['Compliance']

File 20190703_삼성전자_5302

File 20190710_CJ프레시웨이_531020.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_GKL_531094.txt [ 1 ] Line Searching : ['Compliance']




File 20190710_JYPEnt_531089.txt [ 1 ] Line Searching : ['Compliance']


File 20190710_KTG_531006.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_KTG_531103.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_LG화학_531144.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_NAVER_531038.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_NAVER_531119.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_NAVER_531135.txt [ 1 ] Line Searching : ['Compliance', 'Compliance', 'Compliance', 'Compliance', 'Compliance']

File 20190710_NHN_531139.txt [ 1 ] Line Searching : ['Compliance', 'Compliance', 'Compliance', 'Compliance', 'Compliance']

File 20190710_S-Oil_531142.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_SK이노베이션_531141.txt [ 1 ] Line Searching : ['Compliance']

File 20190710_SK텔레콤_531091.txt [ 1 ] Line Search

File 20190715_종근당_531625.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_카카오_531501.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_카카오_531628.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_코스맥스_531489.txt [ 1 ] Line Searching : ['Compliance', 'Compliance', 'Compliance', 'Compliance']

File 20190715_코스맥스_531537.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_코스메카코리아_531493.txt [ 1 ] Line Searching : ['Compliance', 'Compliance', 'Compliance', 'Compliance']

File 20190715_클리오_531492.txt [ 1 ] Line Searching : ['Compliance', 'Compliance', 'Compliance', 'Compliance']

File 20190715_팬오션_531602.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_펄어비스_531542.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_하나투어_531543.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_한국가스공사_531646.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_한국금융지주_531656.txt [ 1 ] Line Searching : ['Compliance']

File 20190715_한국자산신탁_531550.txt [ 25 ] Line 

File 20190722_현대모비스_532286.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_DB손해보험_532481.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_KB금융_532462.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_KCC_532509.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_LG이노텍_532470.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_NH투자증권_532468.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_NH투자증권_532517.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_SK이노베이션_532495.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_SK텔레콤_532482.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_나무가_532412.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_대림산업_532449.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_대한항공_532511.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_덕산네오룩스_532501.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_도이치모터스_532533.txt [ 1 ] Line Searching : ['Compliance']

File 20190723_도이치모터스

File 20190726_더블유게임즈_533013.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_두산_532907.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_두산_532938.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_두산_533103.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_두산중공업_533095.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_롯데정밀화학_532888.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_롯데정밀화학_532908.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_롯데정밀화학_532940.txt [ 1 ] Line Searching : ['Compliance']


File 20190726_만도_532889.txt [ 1 ] Line Searching : ['Compliance']


File 20190726_만도_532943.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_만도_532953.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_만도_532962.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_만도_532990.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_만도_533032.txt [ 1 ] Line Searching : ['Compliance']

File 20190726_만도_533035.txt [ 1 ] Line Se

File 20190731_LG전자_533670.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_OCI_533571.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_SK머티리얼즈_533583.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_골프존_533616.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_녹십자_533550.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_녹십자_533614.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_녹십자_533663.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533508.txt [ 1 ] Line Searching : ['Compliance']



File 20190731_대림산업_533572.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533584.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533612.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533624.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533651.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533662.txt [ 1 ] Line Searching : ['Compliance']

File 20190731_대림산업_533669.txt [ 1 

In [ ]:
# Compliance관련 내용은 있는데 Compliance 단어가 없다
for file in text_list:
    remove_Compliance(file)